In [ ]:
from IPython.core.display import HTML, display
HTML("""
<style>
svg {
    width:40% !important;
    height:40% !important;
}

.container { 
    width:100% !important;
}
</style>
""")

In [ ]:
# %autosave 0
%config IPCompleter.greedy=True

In [ ]:
import chess

## Evaluieren des Schachbretts & Berechnen attackierter Figuren

Um mögliche Züge der KI bewerten zu können und den besten Zug auszuwählen, müssen diese einen Wert zugewiesen bekommen, der Aussage über die Qualität des Zuges gibt. Dafür können unter anderem die Bewertung des Schachbretts durch Summieren zugewiesener Werte für alle lebenden Figuren oder das Berechnen und Einbeziehen attackierter Figuren zur Hilfe genommen werden.

Dabei ist zunächst jedem Figurentyp ein Wert zuzuweisen. Üblicherweise werden Bauern dabei 1 Punkt, Türmen 5 Punkte, Springern sowie Läufern jeweils 3 Punkte und der Dame 9 Punkte zugeordnet.

In [ ]:
PAWN_VALUE = 1
ROOK_VALUE = 5
KNIGHT_VALUE = 3
BISHOP_VALUE = 3
QUEEN_VALUE = 9
KING_VALUE = 20

def assign_piece_value(piece_type):
    return {
        1: PAWN_VALUE,
        2: KNIGHT_VALUE,
        3: BISHOP_VALUE,
        4: ROOK_VALUE,
        5: QUEEN_VALUE,
        6: KING_VALUE
    }[piece_type]

Um den Wert des Schachbretts zu berechnen muss zunächst der Wert aller weißer Figuren berechnet werden und von diesem der Wert aller schwarzen Figuren abgezogen werden. Ist das Ergebnis positiv, verläuft das Spiel günstig für den Spieler der weißen Figuren. Ist das Ergebnis negativ, liegt der Spieler der schwarzen Figuren im Vorteil. Umso höher der Betrag des jeweiligen Spielers ist, desto mehr liegt dieser im Vorteil.

Damit die Werte der Spieler berechnet werden können, wird die Anzahl aller Figurentypen, der jeweiligen Farbe, berechnet und diese mit dem Wert der Figurentypen multipliziert. Am Ende werden die Ergebnisse für alle Figurentypen zusammengezählt und zurückgegeben.

In [ ]:
def get_value_by_color(color):
    attacked_pieces_value = map(lambda piece_type : len(board.pieces(piece_type, color)) * assign_piece_value(piece_type), chess.PIECE_TYPES)
    return sum(attacked_pieces_value)

def get_board_value():
    white_value = get_value_by_color(chess.WHITE)
    black_value = get_value_by_color(chess.BLACK)
    
    return white_value - black_value

Die Berechnung der attackierten Figuren ist ähnlich zu dem Vorgehen bei der Berechnung des Brettwerts. Dabei werden erst die Werte, der vom weißen Spieler attackierten Figuren berechnet und davon die Werte der vom schwarzen Spieler attackierten Figuren abgezogen. Auch hierbei ist ein positives Ergebnis zum Vorteil des Spielers der weißen Figuren und ein negatives Ergebnis zum Vorteil des Spielers der schwarzen Figuren. Ebenso gilt umso höher der Wert, desto deutlicher der Vorteil.

Um diese Werte der attackierten Figuren zu berechnen wird jedes Feld durchgegangen. Daraus werden die Felder gefiltert, die von einer Figur der Farbe des Verteidigers belegt sind und von einer Figur der angreifenden Farbe attackiert werden können. Anschließend wird zu diesen Feldern der Typ der Figur zugeordnet, die sich auf dem Feld befindet. Daraufhin werden diesen ihre jeweiligen Werte zugeordnet und diese abschließend summiert.

In [ ]:
def get_attacked_pieces_value_by_color(attacker_color, defender_color):
    # filters squares for attacked squares, on which a figure of defender is placed
    attackedSquares = filter(lambda square : board.is_attacked_by(attacker_color, square) and not board.piece_at(square) is None and board.piece_at(square).color is defender_color, chess.SQUARES)
    # maps piece type to attacked figure
    attackedPieces = map(lambda square : board.piece_at(square).piece_type, attackedSquares)
    # maps piece value to attacked pieces
    value = map(assign_piece_value, attackedPieces)
    # sums piece value of all attacked pieces
    return sum(value)
     
def get_attacked_pieces_value():
    white_value = get_attacked_pieces_value_by_color(chess.WHITE, chess.BLACK)
    black_value = get_attacked_pieces_value_by_color(chess.BLACK, chess.WHITE)
    
    return white_value - black_value

Im folgenden Codeausschnitt wird ein Beispiel gegeben wie durch die Verwendung, der zuvor vorgestellten Funktionen, eine Auskunft über den akutellen Spielstand gegeben werden kann.

In [ ]:
board = chess.Board()

board_value = get_board_value()
attacked_pieces_value = get_attacked_pieces_value()

# compute the player with a higher value of pieces
if board_value > 0:
    print("White held the lead ahead of Black")
elif board_value < 0:
    print("Black held the lead ahead of White")
else:
    print("Draw")
    
print("Value of the attacked pieces: ", attacked_pieces_value)

## Erstellen des Spiels und der Spieler

Vor dem Start eines Schachspiels müssen zunächst Spiel generell sowie die partizpierenden Spieler der gewünschten Spielertypen erstellt werden. Dazu muss der Spieler dem Programm entweder Parameter mitgeben, die das Spiel direkt mit den gewünschten Spielern startet, oder aber der Spieler wird zum Programmstart nach gewünschten Einstellungen gefragt.

In beiden Fällen wird anschließend eine Liste an Spielern an Hand der gegebenen Funktionen erstellt, mittels welcher Das Spiel dann stattfinden kann. Dies findet in der Funktion `interrogate_settings` statt.

Wird das Spiel mit Parametern gestartet werden diese mit in die `interrogate_settings` Funktion übermittelt. Andernfalls werden alle fehlenden Informationen in der Funktion über die Nutzerschnittstelle abgefragt. Dies sieht wie folgt aus:

In [ ]:
def interrogate_settings(self, player_names=None, player_types=None, player_difficulty = None):
    players = []
    for i in range(2):
        num = i+1
        if player_names is None:
            name = self.get_player_name(num)
        else:
            name = player_names[i]

        if player_types is None:
            player_type = self.get_player_type(num)
        else:
            player_type = player_types[i]

        difficulty = None
        if player_type == 2 and player_difficulty is None:
            difficulty = self.get_difficulty(num)
        elif player_type == 2 and player_difficulty is not None:
            difficulty = player_difficulty[i]

        new_player = PlayerSettings(num, name, player_type, difficulty)
        players.append(new_player)

    return players

Dabei werden für jeden Spieler einmal ein Block an Abfragen durchgegangen, der alle Einstellungen abfragt und diese entsprechend speichet.

Nach der Abfrage der Informationen "Name" und "Spielertyp" wird im Falle, dass dieser Spielertyp "2", der der KI entspricht, gleicht, auch noch der Schwierigkeitsgrad abgefragt, sollte dieser nicht bereits als Startparameter übergeben wordne sein.

Anschließend wird ein neues Einstellungsparameterschema für Spieler, genannt `PlayerSettings`, erstellt und der Liste hinzugefügt. Diese Liste wird nach Durchgang der Schleife für beide Spieler zurück gegeben.

Nachdem die Einstellungen für alle Spieler bekannt sind, werden die Spieler erstellt. Dies funktioniert wie in nachfolgendem Code-Snippet zu sehen.

In [ ]:
player_settings = settings_ui.interrogate_settings()
players = []
for player_setting in player_settings:
    type = type_switcher(player_setting.type)
    players.append(type.Player(player_setting.num, player_setting.name, ui_status, player_setting.difficulty))

Dabei werden zunächst wie gehabt die Einstellungen abgefragt, ehe eine Liste aller Spieler erstellt wird. Dann wird über jeden Eintrag in der Einstellungsliste iteriert und für diesen zunächst der Spielertyp berechnet. Dabei wird der `type_switcher` zur Hilfe genommen, welcher je nach angegebenen Typ die entsprechende Klasse des Spielertyps zurückgibt. Mittels dieser wird dann ein neuer Spieler erstellt, wobei beim Initialisierren alle weiteren Daten mitgegeben werden. Dieser neue Spieler wird dann der Spielerliste hinzugefügt. Mit dieser Liste an Spielern wird dann der `ChessMaster` initialisiert, der im nächsten Kapitel beschrieben wird.

## Verwalten des Schachspiels und Pflege des Spielverlaufs

Ein essentieller Part im Erstellen eines Schachprogrammes ist erstmal das Verwalten des Schachspiels. Dabei muss garantiert werden, dass
- Solange das Spiel nicht vorbei ist, die Spieler abwechselnd einen Zug auswählen können
- Der Zug auf dem vorhandenen Schachbrett durchgeführt wird
- Das daraus entstehende Schachbrett dem Spieler sichtbar gemacht wird

Dies ist Aufgabe des `ChessMaster` und ist mit folgendem Code umgesetzt

In [ ]:
board = chess.Board()
while not board.is_game_over():
    current_player = players[int(not board.turn)]
    current_player.print_board(current_player.name, board)

    move = current_player.get_move(board)
    board.push(move)
    current_player.submit_move(move)

Dabei wird zunächst ein neues Schachbrett - hier board genannt - erstellt. Solange das Spiel auf diesem nicht vorbei ist, was mittels der `board.is_game_over()` Funktiong geprüft werden kann, wird dann stets die gleiche Schleife durchlaufen.

In dieser wird zuallererst der aktuelle Spieler ermittelt und referenziert. Dazu wird aus einer vorhandenen Liste aller Spieler derjenige gewählt, dessen Position in der Liste der gespeicherten Zugnummer des boards entspricht. Diese ist entweder 0, wenn der weiße Spieler an der Reihe ist, oder 1, wenn der schwarze Spieler den nächsten Zug auswählen kann. 

Der daraus berechnete Spieler wurde zuvor beim Spielstart ein Spielertyp zugewiesen, der auf einer Schnittstelle basiert und somit alle nötigen Funktionen implementiert. Diese wurden im Kapitel INSERT beschrieben. Im Verwalter des Schachspiels werden diese nun nach für nach aufgerufen.

Zunächst wird das board für den Spieler ausgegeben mittels der `print_board` Funktion. Ist der Spieler ein Nutzer, so wird diese für gewöhnlich am Nutzerinterface ausgegeben. Andernfalls ist dies nicht nötig und die Funktion kann leer bleiben, ohne etwas auszugeben.

Nun kommt es zum wichtigsten Teil - dem Berechnen des nächsten Zuges. Dazu wird der Spieler aufgefordert an Hand eines gegebenen boards den nächstne Zug zu nennen. In diesem Teil übernimmt beispielsweise die KI ihre Berechnungen für den nächsten Schachzug. Der Nutzer dagegen gibt diesen mittels eines Eingabefeldes ein. 

Nachdem der Zug vom entsprechenden Spieler berechnet und zurückgegeben wurde, wird dieser dem aktuellen board hinzugefügt. Dadurch wechselt auch automatisch der Spieler, der an der Reihe ist, wodurch dieser im nächsten Schleifendurchlauf nach dessen Zug gefragt wird.

Abschließend wird das neue board nochmal abgeschickt. Dabei kann zum Beispiel eine erneute Ausgabe des Schachbretts mit dem aktualisierten Zustand stattfinden oder beispielsweise bei der Nutzung einer Online-API der gewählte Zug an die Schnittstelle gesendet werden.

Nach dem Ende des Spiels wird noch das Ergebnis ausgegeben. Der Code dazu ist im folgenden Code Snippet zu sehen.

In [ ]:
result = Tools.get_board_result(board)
if result is 1:
    players[0].print_win_msg()
    players[1].print_loose_msg()
elif result is -1:
    players[1].print_win_msg()
    players[0].print_loose_msg()
else:
    players[0].print_draw_msg()
    players[1].print_draw_msg()

Dabei wird zuerst das Ergebnis an Hand der `get_board_result` Funktion aus der Hilfsklasse "Tools" ermittelt. Diese sieht wie folgt aus:

In [ ]:
if board.is_variant_loss():
    return -1 if board.turn == chess.WHITE else 1
elif board.is_variant_win():
    return 1 if board.turn == chess.WHITE else -1
elif board.is_variant_draw():
    return 0
# Checkmate.
if board.is_checkmate():
    return -1 if board.turn == chess.WHITE else 1
# Draw claimed.
if board.can_claim_draw():
    return 0
# Seventyfive-move rule or fivefold repetition.
if board.is_seventyfive_moves() or board.is_fivefold_repetition():
    return 0
# Insufficient material.
if board.is_insufficient_material():
    return 0
# Stalemate.
if not any(board.generate_legal_moves()):
    return 0
return 0

Dabei wird jede mögliche Option, wie das Spiel zum Ende gekommen sein kann, durchgegangen und anschließend das jeweilige Ergebnis zurückgegeben. Eine 1 steht für einen Sieg für weiß, eine -1 für einen Sieg für schwarz und eine 0 für ein Unentschieden beziehungsweise einen Patt.

Nach Abfrage des Ergebnisses werden beide Spieler abhängig von diesem entweder dazu aufgefordert eine Siesesbenachrichtigung auszugeben oder aber eine Benachrichtigung über die Niederlage oder ein Unentschieden.

Zusätzlich zu dem Verwalten des Spiels ist es auch Aufgabe des `ChessMaster`s die Historie aller Spiele zu pflegen. Dazu wird zunächst eine Liste von Schachbrettern der Klasse `chess.Board` angelegt. Nach jedem durchgeführten Zug wird der neue Zustand des boards zu dieser hinzugefügt. Dabei wird das Schachbrett in der fen-Notation gespeichert, die in einem String den exakten Zustand des Schachbretts wiedergeben kann. Dabei wird jedoch nur der erste Teil dieser Notation gespeichert, da dieser alleine bereits Aufschluss über die Positionierungen gibt. Die darauf folgenden Teile sind zum Speichern des Spielers, der am Zug ist, wie viele Züge bereits durchgeführt wurden und weitere Informationen, die zum Bewerten in der Historie nicht notwendig sind. 

Das Speichern der Züge im Verlauf des Spiels sieht dann wie folgt aus

In [ ]:
board = chess.Board()
turn_list = list()

while not board.is_game_over():
    current_player = players[int(not board.turn)]
    current_player.print_board(current_player.name, board)

    move = current_player.get_move(board)
    board.push(move)
    current_player.submit_move(move)
    
    turn_list.append(board.fen().split(" ")[0])

In Zeile 2 wird die Liste erstellt und in der letzten Zeile der Zustand in diese Liste gespeichert. Nach Ende des Spiels muss diese Liste noch in die bestehende Historie eingepflegt werden. Dies ist in folgendem Code-Snippet zu sehen.

In [ ]:
def groom_board_history(self, final_board, turn_list):
    victory_status = Tools.get_board_result(final_board)

    new_turn_dict = dict.fromkeys(turn_list, victory_status)

    history = pd.read_csv(HISTORY_FILE_LOC)
    history_dict = dict(zip(list(history.board), list(history.value)))

    merged_history_dict = { k: new_turn_dict.get(k, 0) + history_dict.get(k, 0) for k in set(new_turn_dict) | set(history_dict) }
    merged_history = pd.DataFrame(list(merged_history_dict.items()), columns=['board','value'])

    merged_history.to_csv(HISTORY_FILE_LOC)

Dabei wird zuerst das Ergebnis ermittelt mittels der bereits vorgestellten Funktion `get_board_result`. Danach wird ein Dictionary angelegt, das auf der einen Seite jeden gespeicherten Zustand des einzupflegenden Spiels als Schlüssel enthält und auf der anderen Seite als Wert den Ausgang des Spiels. Zur Erinnerung - dieser beträgt 1 bei Sieg von weiß, 0 bei Unentschieden/Patt und -1 bei Sieg von Schwarz.

Nachdem das Dictionary der neu einzupflegenden Züge angelegt ist, wird die vorhandene Historie aus der entsprechenden Datei ausgelesen. Dies wird mittels der in `panda` enthaltenden Funktion `pd.read_csv` durchgeführt. Anschließend wird auch daraus ein Dictionary erstellt, indem die Zeile "board" als Schlüssel und die Zeile "value" als Wert verwendet wird.

Nun werden die beiden Dictionaries zusammengefügt. Dazu wird für jeden Schlüssel aus dem Dictionary der neu einzupflegenden Spielzüge oder dem vorhandenen Historie-Dictionary der berechnete Wert aus ersterem auf den vorhandenen Wert in der Spielhistorie aufaddiert. Dadurch erhalten wird ein Dictionary, das alle Schachzustände aus der Historie sowie den neuen Spielzuständen vereint und dessen Werte durch Addition kombiniert, wodurch der neu berechnete Wert Aufschluss über den wahrscheinlichen Sieger ausgehend von einem bestimmten Zustand geben kann.

Anschließend wird dieses Dictionary in ein Panda Dataframe umgewandelt, wobei der Schlüssel für die Zeile "board" und der Wert für die Zeile "value" verwendet wird, um das DataDrame anschließend wieder zu der CSV-Datei zu speichern.

Dies ermöglicht nicht nur eine Verwaltung des Spiels, sondern gleichzeitig auch eine Speicherung aller möglichen Zustände, die bei den Evaluierungsfunktionen (beschrieben in INSERT) zur Hilfe genommen werden können

## Implementierung des "Iterative Deepening"

Die zentrale Aufgabe der KI ist es, den best möglichen Zug für eine gegebene Situation zu finden. Dabei geht diese so vor, dass sie alle möglichen Züge durchgeht und die daraus entstehenden Zustände analysiert und evaluiert. Dabei jedoch zählt nicht nur der direkt erreichbare Zustand zählt, sondern auch die aus diesem Zustand erreichbaren Zustände und so weiter. Aus diesem Grund wird immer bis zu einer bestimmten Tiefe in die Züge hineingeschaut und die sich daraus ergebenden Zustände evaluiert.

Um dies jedoch nicht fest immer bis zu einer bestimmten Tiefe durchgehen zu lassen, sondern variabel anzupassen, je nachdem wie viele Züge von dem gegebenen Zustand aus möglich sind, kann ein Zeitlimit dienen. Dabei wird anfangs die Tiefe auf 1 gesetzt und dann mittels des Minimax-Algorithmus die Züge evaluiert. Danach wird die Tiefe um 1 erhöht und erneut der Minimax-Algorithmus angewandt. Dies wird solange wiederholt, bis die angegebene Zeit abgelaufen ist. Dieser Algorithmus nennt sich "Iterative Deepening". Nähere Informationen dazu sind in Kapitel INSERT aufgeführt. 

Dem Algorithmus muss dazu der aktuelle Zustand sowie eine maximale Tiefe mitgegeben werden. Ist diese erreicht bricht der Algorithmus ab, unabhängig davon, ob das Zeitlimit überschritten ist oder nicht. Zunächst müssen beim Ausführen dann einige Werte wie folgt festgelegt werden.

In [ ]:
depth = 1
    
start_time = int(time.time())
end_time = start_time + self.time_limit
current_time = start_time

player = bool(board.turn)
self.best_possible_result = self.get_best_possible_result(board, player)

Zunächst wird die Starttiefe auf 1 festgesetzt. Danach wird die Startzeit auf die aktuelle Zeit gesetzt und die Endzeit berechnet, indem auf die Startzeit das Zeitlimit addiert wird. Zudem wird der erste Wert für die aktuelle Zeit auf die Startzeit festgelegt.

Anschließend wird für den Spieler, der aktuell am Zug ist, berechnet, was das bestmöglich zu erreichende Resultat ist. Dies wird mit der Funktion `get_best_possible_result` durchgeführt. Dies ist dazu gut, um finale Zustände dahingehend zu evaluieren, ob diese für den Nutzer die best mögliche Option ist (Sieg oder Unentschieden wenn Sieg nicht mehr möglich ist) und dementsprechend zu bewerten. Die Funktion sieht wie folgt aus.

In [ ]:
def get_best_possible_result(self, board, player):
    if player and board.has_insufficient_material(chess.WHITE):
        return 0
    if not player and board.has_insufficient_material(chess.BLACK):
        return 0
    if player and not board.has_insufficient_material(chess.WHITE):
        return 1
    if not player and not board.has_insufficient_material(chess.BLACK):
        return -1

Dabei muss der Funktion der aktuelle Zustand sowie der Spieler, der an der Reihe ist, mitgegeben werden. Ist der aktuelle Spieler der der weißen Figuren (player == True) und hat weiß unzureichende Materialien für einen Sieg, so ist der bestmögliche Zustand ein Patt. Das gleiche Ergebnis wird zurückgegeben, wenn der Spieler der der schwarzen Figuren ist (player == False) und schwarz unzureichende Materialien hat.

Ist der Spieler jedoch weiß und er hat noch ausreichend Materialien, so wird der Wert 1 zurückgegeben, da ein Sieg noch erreichbar ist. Genauso wird für den schwarzen Spieler der Wert -1 zurückgegeben, falls er noch ausreichende Materialien besitzt, da dieser noch einen Sieg erreichen kann und der Wert -1 für einen Sieg von Schwarz steht.

Nach dieser Abfrage wird der eigentliche Algorithmus des "Iterative Deepening" durchgeführt.

In [ ]:
legal_moves = list(board.legal_moves)
while current_time < end_time and depth <= max_depth:
    move_val_dict = {}

    best_value = float('-inf')
    best_move = legal_moves[0]

    for move in legal_moves:
        tmp_board = chess.Board(str(board.fen()))
        tmp_board.push(move)

        value = self.min_value(str(tmp_board.fen()), player, float('-inf'), float('inf'), depth - 1, end_time)
        move_val_dict[move] = value

        if value == MAX_BOARD_VALUE:
            return move
        if value > best_value:
            best_value = value
            best_move = move

    legal_moves.sort(key=move_val_dict.get, reverse=True)
    depth += 1
    current_time = int(time.time())

Dabei wird zunächst eine Liste aller legalen Züge erstellt. Dann wird eine Schleife so lange durchlaufen, bsi entweder die Zeit abgelaufen ist oder aber die maximale Tiefe erreicht ist. 

In dieser Schleife wird ein Dictionary aller Züge mit ihren berechneten Werte erstellt. Zudem werden Anfangswerte für die besten Züge und dessen Wert festgelegt. Der Anfangswert des besten Zugs wird auf den ersten Zug festgesetzt. Der Wert dieses wird auf den Wert "- Unendlich" gesetzt.

Nun wird über alle legalen Züge iteriert und für jeden Zug ein temporärer "board" angelegt, das erreichbaren Zustand widerspiegelt. Nun wird mittels des Minimax-Algorithmus der Wert dieses boards ermittelt. Dabei wird als Alpha "- Unendlich" und als Beta "Unendlich" mitgegeben. Was die Werte Alpha und Beta aussagen, wird im Kapitel INSERT beschrieben. Zudem wird die Tiefe auf einen Wert festgelegt, der um einen Wert geringer ist als die maximale Tiefe, da durch Aufruf der Funktion in die erste Tiefe hineingegangen wurde. Zudem wird die Zeit mitgegeben, zu der der Algorithmus enden muss, damit der Minimax Algorithmus dementsprechend endet und das zeitlimit nicht überschreitet.

Nachdem der Minimax-Algorithmus fertig durchlaufen ist, wird der Wert mit dem Zug zu dem Dictionary hinzugefügt. Gleicht der Wert dem maximalen Wert für einen Zustand, ist also dementsprechend ein Sieg, wird der Zug direkt zurückgegeben, da mit diesem dann auf jeden Fall ein Sieg erreich werdne kann. Andernfalls wird der Wert verglichen, ob er besser ist als der aktuelle Wert. Ist dies der Fall, so wird der neue beste Wert auf den aktuell berechneten festgelegt, ebenso wie der beste Zug auf den der aktuellen Iteration gesetzt wird.

Nachdem alle Züge durchlaufen wurden, wird die Liste aller legalen Züge an Hand der berechneten Werte sortiert, damit im nächsten Durchlauf die Züge in dieser Reihenfolge durchlaufen werden. Dies verbessert den Durchsatz beim Alpha Beta Pruning, wie in Kapitel INSERT beschrieben. 

Abschließend wird noch die Tiefe um 1 erhöht und die aktuelle Zeit auf die Systemzeit gesetzt, damit an Hand dieser entschieden werden kann, ob der Algorithmus noch weiter durchlaufen darf.

Nachdem dann die Zeit abgelaufen ist und alle Züge in der für die angegebenen Zeit maximalen Tiefe evaluiert wurden, wird der best mögliche Zug zurückgegeben. Dieser wird dann von der Ki ausgeführt.

## Implementierung des Minimax-Algorithmus mit Alpha-Beta-Pruning

## Evaluierung eines gegebenen Schachbretts